# Imports and Constants

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import random
from sklearn.feature_selection import SelectKBest, f_classif

from saving_outputs import *
from load_data import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *

C:\Users\nicol\miniconda3\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [6]:
from_who = "our"   # 2 possibilities : "mohameds" or "our"
voxel_size = "3" # 2 possibilities : "2" or "3"
radius_mask = "10" # [mm] from 5 to 11
use_t_maps = True # use t-maps or beta-maps
correction = "_Tcorrected" # a t-value has been used to create ROIs or no

maps_folder="brain_maps/"+from_who+"_maps"+"_"+voxel_size
masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+radius_mask+correction

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
labels = {'vis' : np.array(classes*nb_runs), 'aud' : np.array(classes*nb_runs)}
labels_same = np.array(classes*nb_runs)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                           (["vis"], ["PT_L", "PT_R"]),
                           (["aud"], ["V5_L", "V5_R"]),
                           (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                            (["vis", "aud"], ["PT_L", "PT_R"])]

def k_selector(n_voxel):
    return int(0.7*n_voxel)

std_scaler = sklearn.preprocessing.StandardScaler()

selector = SelectKBest(score_func=f_classif)

classifiers = {
    'svm':sklearn.svm.SVC(random_state=SEED),
    'LR':sklearn.linear_model.LogisticRegression(random_state=SEED),
    'KNN':sklearn.neighbors.KNeighborsClassifier(),
    'perceptron':sklearn.linear_model.Perceptron(random_state=SEED)
    #'ELM':skelm.ELMClassifier(random_state=SEED, n_neurons=100, alpha=1),
    }

param_grids = {
    'svm':{
        'svm__C': [10**x for x in np.linspace(-3,3,num=20)],
        #'svm__gamma': [1, 0.1],
        'svm__kernel': ['linear','rbf','sigmoid']},

    'LR':{
        'LR__C': [10**x for x in np.linspace(-3,3,num=20)]},
    'KNN':{
        'KNN__n_neighbors': [1,3,5,10,20],
        'KNN__weights':["uniform", "distance"],
    },
    'perceptron':{
        'perceptron__penalty':["l1", "l2", "elasticnet", None],
        'perceptron__max_iter':[100,300,1000],
    }
}
cv_scheme = list()
for i in range(11):
    full_idx = range(44)
    idx_te = [i*4,i*4+1,i*4+2,i*4+3]
    idx_tr = [x for x in full_idx if x not in idx_te]
    tr_te_splits = [idx_tr, idx_te]
    cv_scheme.append(tr_te_splits)

models = dict()
for name in classifiers:
    pipeline = Pipeline([('scaler', std_scaler),
                         #('kbest', selector),
                         (name, classifiers[name])])
    GS = GridSearchCV(pipeline, param_grids[name], cv = cv_scheme,
                      n_jobs = 4)
    models[name] = GS
    
decoder = Decoder(n_perm=n_individual_perms, models=models, n_classes=len(classes), n_splits=nb_runs, seed=SEED, verbose=2)

# Loading data

In [7]:
maps_masked, masks_exist = load_full_data(subjects_ids, len(classes), nb_runs, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"), use_t_maps=use_t_maps)
decoder.set_masks_exist(masks_exist)

In [ ]:
n_voxels = maps_masked[0]["vis"][0]["V5_L"].shape[1]
for name in classifiers:
    if isinstance(decoder.models[name].estimator.steps[1][1], SelectKBest): 
        decoder.models[name].estimator.steps[1][1].set_params(k = k_selector(n_voxels))
print("initially "+str(n_voxels)+" voxels, selected = "+str(k_selector(n_voxels))+" voxels.")

# Within-modality decoding

In [8]:
confusion_matrixes_within, val_scores_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
confusion_matrixes_within = change_confusion_matrixes_org(confusion_matrixes_within, subjects_ids, models.keys())
val_scores_within = change_confusion_matrixes_org(val_scores_within, subjects_ids, models.keys())

subject 1/23 done in 227.57413625717163 seconds
subject 2/23 done in 446.1944360733032 seconds
subject 3/23 done in 613.0493385791779 seconds
subject 4/23 done in 856.329986333847 seconds
subject 5/23 done in 1088.665104150772 seconds
subject 6/23 done in 1301.8827946186066 seconds
subject 7/23 done in 1512.4825913906097 seconds
subject 8/23 done in 1724.7149147987366 seconds
subject 9/23 done in 1939.1268103122711 seconds
subject 10/23 done in 2146.771112680435 seconds
subject 11/23 done in 2359.4018857479095 seconds
subject 12/23 done in 2563.850994825363 seconds
subject 13/23 done in 2762.966330051422 seconds
subject 14/23 done in 2972.632229566574 seconds
subject 15/23 done in 3175.932105064392 seconds
subject 16/23 done in 3384.8602752685547 seconds
subject 17/23 done in 3617.176152229309 seconds
subject 18/23 done in 3835.220465898514 seconds
subject 19/23 done in 3995.8625054359436 seconds
subject 20/23 done in 4205.297813415527 seconds
subject 21/23 done in 4258.922575712204 se

# Cross-modal decoding

In [9]:
confusion_matrixes_cross, val_scores_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
confusion_matrixes_cross = change_confusion_matrixes_org(confusion_matrixes_cross, subjects_ids, models.keys())
val_scores_cross = change_confusion_matrixes_org(val_scores_cross, subjects_ids, models.keys())

subject 1/23 done in 206.38288974761963 seconds
subject 2/23 done in 400.86260294914246 seconds
subject 3/23 done in 547.9106330871582 seconds
subject 4/23 done in 753.1565067768097 seconds
subject 5/23 done in 961.1341166496277 seconds
subject 6/23 done in 1172.1722340583801 seconds
subject 7/23 done in 1375.2842104434967 seconds
subject 8/23 done in 1584.5483627319336 seconds
subject 9/23 done in 1809.208894252777 seconds
subject 10/23 done in 2022.4113788604736 seconds
subject 11/23 done in 2236.3046491146088 seconds
subject 12/23 done in 2438.066136598587 seconds
subject 13/23 done in 2644.2617440223694 seconds
subject 14/23 done in 2856.3597161769867 seconds
subject 15/23 done in 3062.58411359787 seconds
subject 16/23 done in 3274.8009011745453 seconds
subject 17/23 done in 3488.6492404937744 seconds
subject 18/23 done in 3701.8641612529755 seconds
subject 19/23 done in 3861.0293905735016 seconds
subject 20/23 done in 4066.3144931793213 seconds
subject 21/23 done in 4117.125281095

# Bootstrapping to assess group-level significance

In [ ]:
n_single_perm = 50
n_bootstrap = 100_000

In [ ]:
within_cf_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same, within_modal_tasks_regions,maps_masked,n_subjects, within_modality=True)
within_cf_100_perm = change_cfm_bootstrap_org(within_cf_100_perm, subjects_ids, models.keys(), n_single_perm)

cross_cf_100_perm = decoder.score_bootstrapped_permutations(n_single_perm, labels_same, cross_modal_tasks_regions,maps_masked,n_subjects, within_modality=False)
cross_cf_100_perm = change_cfm_bootstrap_org(cross_cf_100_perm, subjects_ids, models.keys(), n_single_perm)

In [ ]:
bootstrapped_distribution_cross = dict()
bootstrapped_distribution_within = dict()
for name in models.keys():
    within_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, within_cf_100_perm[name], len(classes))
    bootstrapped_distribution_within[name] = compute_bootstrap_distribution(n_bootstrap, n_subjects, within_scores_100_perm, n_single_perm)

    cross_scores_100_perm = compute_accuracy_bootstrap(n_subjects, n_single_perm, cross_cf_100_perm[name], len(classes))
    bootstrapped_distribution_cross[name] = compute_bootstrap_distribution(n_bootstrap, n_subjects, cross_scores_100_perm, n_single_perm)

# Saving results

In [10]:
type_maps = "_t_maps_" if use_t_maps else "_beta_maps_"
out_directory = "out/"+from_who+type_maps

for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+voxel_size+"_radius="+radius_mask+correction+"/"
    create_directory(out_dir)
    save_dicts(out_dir+"masks_exist.csv", masks_exist, list(masks_exist[0].keys()), subjects_ids)
    
    save_dicts(out_dir+"confusion_matrixes_within.csv", confusion_matrixes_within[name], list(confusion_matrixes_within[name][0].keys()), subjects_ids)
    save_dicts(out_dir+"validation_scores_within.csv", val_scores_within[name], list(val_scores_within[name][0].keys()), subjects_ids)
    acc_within = compute_metric(out_dir, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)

    compute_metric(out_dir, subjects_ids, {'name' : 'recall', 'function':recall}, "within", masks_exist, len(classes))
    compute_metric(out_dir, subjects_ids, {'name' : 'precision', 'function':precision}, "within", masks_exist, len(classes))

    within_modality_group_results = average_dicos(acc_within)
    save_dicts(out_dir+"group_scores_within.csv", [within_modality_group_results], list(within_modality_group_results.keys()), [0])

    save_dicts(out_dir+"confusion_matrixes_cross.csv", confusion_matrixes_cross[name], list(confusion_matrixes_cross[name][0].keys()), subjects_ids)
    save_dicts(out_dir+"validation_scores_cross.csv", val_scores_cross[name], list(val_scores_cross[name][0].keys()), subjects_ids)
    acc_cross = compute_metric(out_dir, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)
    
    compute_metric(out_dir, subjects_ids, {'name' : 'recall', 'function':recall}, "cross", masks_exist, len(classes))
    compute_metric(out_dir, subjects_ids, {'name' : 'precision', 'function':precision}, "cross", masks_exist, len(classes))

    cross_modality_group_results = average_dicos(acc_cross)
    save_dicts(out_dir+"group_scores_cross.csv", [cross_modality_group_results], list(cross_modality_group_results.keys()), [0])

C:\Users\nicol\OneDrive\Bureau\Mémoire\MasterThesisGit\metrics.py:46: RuntimeWarning: invalid value encountered in long_scalars
  recall[i] = confusion_matrix[i][i]/sum([confusion_matrix[j][i] for j in range(n_classes)])


In [ ]:
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+voxel_size+"_radius="+radius_mask+"/"
    save_dicts(out_dir+"accuracy_bootstraps_within.csv", [bootstrapped_distribution_within[name]], list(bootstrapped_distribution_within[name][0].keys()), range(n_bootstrap))
    save_dicts(out_dir+"accuracy_bootstraps_cross.csv", [bootstrapped_distribution_cross[name]], list(bootstrapped_distribution_cross[name][0].keys()), range(n_bootstrap))

    cv_group_df = retrieve_cv_metric(out_dir, "group_scores")
    bootstrap_df  = retrieve_bootstrap_metric(out_dir, "accuracy")
    pvals = compute_p_val_bootstrap(bootstrap_df, cv_group_df)
    save_dicts(out_dir+"estimated_pval_bootstrap.csv", [pvals], list(pvals.keys()), [0])

# Plotting results (from files of saved results)

In [13]:
type_maps = "_t_maps_" if use_t_maps else "_beta_maps_"
out_directory = "out/"+from_who+type_maps
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+voxel_size+"_radius="+radius_mask+correction+"/"
    masks_exist = retrieve_masks_exist(out_dir)
    cv_group_df = retrieve_cv_metric(out_dir, "group_scores")
    cv_df = retrieve_cv_metric(out_dir, "accuracy")
    cfm_df = retrieve_cv_matrixes(out_dir)
    val_scores_df = retrieve_val_scores(out_dir)
    pvals = retrieve_pvals(out_dir, default_keys=cv_df.columns)

    plt_dir = "plots/"+from_who+type_maps+str(classifiers[name])+"_"+voxel_size+"_radius="+radius_mask+correction
    create_directory(plt_dir)
    plotter = Plotter(plt_dir, subjects_ids)
    plotter.plot_cv_score_with_points(cv_df, pvals, chance_level = True)
    plotter.plot_validation_scores_hyper_param(val_scores_df, "C", param_grids["svm"]["svm__C"], masks_exist, chance_level=True, log10_scale=True)

    group_cfm = compute_group_confusion_matrix(cfm_df, subjects_ids)
    plotter.plot_group_confusion_matrix(group_cfm, classes)

No p-values found in directory : out/our_t_maps_SVC(random_state=0)_3_radius=10_Tcorrected/
No p-values found in directory : out/our_t_maps_LogisticRegression(random_state=0)_3_radius=10_Tcorrected/


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


No p-values found in directory : out/our_t_maps_KNeighborsClassifier()_3_radius=10_Tcorrected/


C:\Users\nicol\OneDrive\Bureau\Mémoire\MasterThesisGit\plots.py:259: RuntimeWarning: Mean of empty slice
  score_per_analysis[analysis][key] = np.nanmean(tab, axis=0)


No p-values found in directory : out/our_t_maps_Perceptron()_3_radius=10_Tcorrected/


...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 1656x720 with 0 Axes>

In [ ]:
for name in classifiers:
    out_dir = out_directory+str(classifiers[name])+"_"+voxel_size+"_radius="+radius_mask+"/"
    bootstrap_df  = retrieve_bootstrap_metric(out_dir, "accuracy")
    cv_group_df = retrieve_cv_metric(out_dir, "group_scores")
    pvals = retrieve_pvals(out_dir)
    
    plotter.plot_bootstrap(bootstrap_df, cv_group_df, pvals, 30)

# Comparing classifiers

In [2]:
subjects_ids = range(1,24)
out_folders = ["out/our_t_maps_SVC(random_state=0)_3_radius=10_Tcorrected/",
               "out/our_t_maps_LogisticRegression(random_state=0)_3_radius=10_Tcorrected/",
               "out/our_t_maps_KNeighborsClassifier()_3_radius=10_Tcorrected/",
               "out/our_t_maps_Perceptron()_3_radius=10_Tcorrected/",
               ]
classifiers_names = ["SVM", "LR", "KNN", "Perceptron"]
plotter = Plotter("plots/comparing_classifiers_"+"_".join(classifiers_names), subjects_ids)
plotter.plot_tests_scores_from_different_folders(out_folders, classifiers_names, "Comparing classifiers", "Classifier")

<Figure size 2880x720 with 0 Axes>

<Figure size 2880x720 with 0 Axes>